In [1]:
from vietocr.tool.config import Cfg
from vietocr.tool.translate import build_model
from vietocr.model.trainer import Trainer

import torch
import torch.nn as nn

In [2]:
config = Cfg.load_config_from_name("vgg_transformer")
config['cnn']['pretrained']=False
config['device'] = 'cpu'
config['pretrain'] = "./ckpts/vgg_transformer.pth"
config['dataloader']['num_workers'] = 0
config['trainer'].update({'num_epochs': 100})
config.update({'slim': {'mode': 'prune'}})

dataset_params = {
    "name": "hw",
    "data_root": "./data_line/",
    "train_annotation": "train_line_annotation.txt",
    "valid_annotation": "test_line_annotation.txt",
}
config['dataset'].update(dataset_params)
config

{'vocab': 'aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0123456789!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ',
 'device': 'cpu',
 'seq_modeling': 'transformer',
 'transformer': {'d_model': 256,
  'nhead': 8,
  'num_encoder_layers': 6,
  'num_decoder_layers': 6,
  'dim_feedforward': 2048,
  'max_seq_length': 1024,
  'pos_dropout': 0.1,
  'trans_dropout': 0.1},
 'optimizer': {'max_lr': 0.0003, 'pct_start': 0.1},
 'trainer': {'batch_size': 32,
  'print_every': 200,
  'valid_every': 4000,
  'iters': 100000,
  'export': './weights/transformerocr.pth',
  'checkpoint': './checkpoint/transformerocr_checkpoint.pth',
  'log': './train.log',
  'metrics': None,
  'num_epochs': 100},
 'dataset': {'name': 'hw',
  'data_root': './data_line/',
  'train_annotation': 'train_line_annotation.txt',
  'valid_annotation': 'test_line_annotation.txt',
  'image_height': 32,
  'im

In [3]:
trainer = Trainer(config=config)

/Users/lihardingnguyen/.miniconda3/envs/ai-env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Before pruning
base_macs=8320363639.28125, base_params=37650217


/Users/lihardingnguyen/.miniconda3/envs/ai-env/lib/python3.11/site-packages/torch_pruning/pruner/algorithms/metapruner.py:101: UserWarning: channel_groups is deprecated. Please use in_channel_groups and out_channel_groups instead.
  warnings.warn("channel_groups is deprecated. Please use in_channel_groups and out_channel_groups instead.")


After pruning
base_macs_after_pruning=2601952009.65625, base_params_after_pruning=13010359
train_hw exists. Remove folder if you want to create new dataset


train_hw build cluster: 100%|███████████████████████████████| 5482/5482 [00:00<00:00, 228844.73it/s]

valid_hw exists. Remove folder if you want to create new dataset



valid_hw build cluster: 100%|███████████████████████████████| 1812/1812 [00:00<00:00, 164084.78it/s]


In [4]:
trainer.train()

/Users/lihardingnguyen/.miniconda3/envs/ai-env/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


iter: 000000 - train loss: nan - lr: 1.20e-05 - load time: 0.13 - gpu time: 0.00


RuntimeError: The size of tensor a (128) must match the size of tensor b (129) at non-singleton dimension 1

In [ ]:
train_loader = trainer.train_gen
for batch in train_loader:
    print(type(batch))
    break

In [ ]:
dummy_input = (batch['img'], batch["tgt_input"], None)
oto = OTO(model=model, dummy_input=dummy_input, strict_out_nodes=True)

In [ ]:
oto.visualize(out_dir="./cache", view=False, display_params=True)

In [ ]:
print(batch['img'].shape, batch["tgt_input"].shape, batch["tgt_padding_mask"].shape)

In [ ]:
for name, m in model.transformer.named_modules():
    if isinstance(m, nn.MultiheadAttention):
        print(name, m)

In [ ]:
optimizer = oto.hesso(
    variant="adamw",
    lr=0.01,
    weight_decay=1e-4,
    target_group_sparsity=0.5,
    start_pruning_step=10 * len(train_data),
    pruning_periods=10,
    pruning_steps=10 * len(train_data),
)

In [ ]:
for m in model.modules():
    if isinstance(m, torch.nn.Linear) and m.out_features == 233:
        ignored_layers.append(m)
        
ignored_layers

In [ ]:
channel_groups = {}
for m in model.modules():
    if isinstance(m, nn.MultiheadAttention):
        channel_groups[m] = m.num_heads
channel_groups

In [ ]:
iterative_steps = 6
example_inputs = (batch["img"], batch["tgt_input"], batch["tgt_padding_mask"])
pruner = tp.pruner.MagnitudePruner(
    model,
    example_inputs,
    importance=imp,
    global_pruning=True,
    iterative_steps=iterative_steps,
    pruning_ratio=0.5, # remove 50% channels, ResNet18 = {64, 128, 256, 512} => ResNet18_Half = {32, 64, 128, 256}
    ignored_layers=ignored_layers,
)

In [ ]:
base_macs, base_nparams = tp.utils.count_ops_and_params(model, example_inputs)

In [ ]:
for i in range(iterative_steps):
    # Soft Pruning
    for group in pruner.step(interactive=True):
        for dep, idxs in group:
            target_layer = dep.target.module
            pruning_fn = dep.handler
            if pruning_fn in [tp.prune_conv_in_channels, tp.prune_linear_in_channels]:
                target_layer.weight.data[:, idxs] *= 0
            elif pruning_fn in [tp.prune_conv_out_channels, tp.prune_linear_out_channels]:
                target_layer.weight.data[idxs] *= 0
                if target_layer.bias is not None:
                    target_layer.bias.data[idxs] *= 0
            elif pruning_fn in [tp.prune_batchnorm_out_channels]:
                target_layer.weight.data[idxs] *= 0
                target_layer.bias.data[idxs] *= 0
                
    macs, nparams = tp.utils.count_ops_and_params(model, example_inputs)
    print(model)
    print(model(*example_inputs).shape)
    
    print(
            "  Iter %d/%d, Params: %.2f M => %.2f M"
            % (i+1, iterative_steps, base_nparams / 1e6, nparams / 1e6)
        )
    print(
            "  Iter %d/%d, MACs: %.2f G => %.2f G"
            % (i+1, iterative_steps, base_macs / 1e9, macs / 1e9)
        )